In [5]:
import sys; sys.path.append('..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer
linkage_name = '20190605_145614_linemodel'
l = elastic_rods.RodLinkage('../../examples/florin/{}.obj'.format(linkage_name), 8)
#driver=166
driver=l.centralJoint()

#mat = elastic_rods.RodMaterial('+', 2000, 0.3, [5, 5, 0.7, 0.7], stiffAxis=elastic_rods.StiffAxis.D1)
mat = elastic_rods.RodMaterial('rectangle', 20000, 0.3, [12, 8])
l.setMaterial(mat)

with suppress_stdout(): elastic_rods.restlen_solve(l)
jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
fixedVars.append(jdo + 6) # constrain angle at the driving joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)

view = LinkageViewer(l, width=1024)
#view.setCameraParams(((2.656451698624758, 1.5087349593639046, 0.23168476156681694),
# (0.052583904614345374, 0.026224725204501566, 0.9982721055721517),
# (0.13680349418553736, -0.13680349418553736, -0.8425278443781967)))
view.show()

Renderer(camera=PerspectiveCamera(aspect=2.0, children=(DirectionalLight(color='white', intensity=0.6, positio…

In [5]:
l.saveVisualizationGeometry('deployed_{}.obj'.format(linkage_name))
#l.writeLinkageDebugData('')

In [4]:
from write_render_files import writeRenderFiles, writeActuators
writeRenderFiles(l, 'Pavilion3s', 'pav3s')
#writeActuators(l, np.where(torques > 1e-4)[0], directory='AsymPointyDataFlat', name='asymPointy_Fab')

In [10]:
from linkage_utils import writeRodSegments
writeRodSegments(l,'rod_segments_{}.txt'.format(linkage_name), zeroBasedIndexing=True)
np.savetxt('restlen_{}.txt'.format(linkage_name),l.getPerSegmentRestLength())
np.savetxt('normals_{}.txt'.format(linkage_name), np.array([j.normal for j in l.joints()]))
l.writeLinkageDebugData('deployed_{}.msh'.format(linkage_name))

In [41]:
# Compute maximum "stress" appearing at any rod vertex in the structure
max([max(segmentStress) for segmentStress in l.rodStresses()[1]])

24.37145054902055

In [ ]:
# Output actuation torque
for forceScale in np.linspace(1.0, 0, 51):
    name = 'Pavilion14_{}'.format(forceScale)
    with suppress_stdout(): elastic_rods.compute_equilibrium(l, forceScale * externalForces, fixedVars=fixedVars)
    view.update(preserveExisting=True)
    writeRenderFiles(l, directory, name)
    writeActuators(l, np.where(torques > 0.1)[0], directory, name)

In [44]:
# Compute average "stress" appearing over all rod vertices in the structure
np.mean(sum(l.rodStresses()[1], []))

11.81760497978405

In [6]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.beta = 1e-8
    opts.gradTol = 1e-4
    opts.useIdentityMetric = False
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)
# open_linkage(l, driver, 2 * np.pi/3, 25, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=False);
open_linkage(l, driver, np.deg2rad(115) - l.averageJointAngle, 50, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=True);

target angle:  0.08281079209860974
0	207019	81214.5	81214.5	1	0
10	101050	1620.01	1620.01	1	0
20	100832	1.87657	1.87657	1	0
0.023129862894894278	100832.35837709754	100832.35837709755	2218.8286100122523	98362.75812704599	250.77164003929258
target angle:  0.1220825849087091
0	135595	41053.7	41053.7	1	0
0.03364608406019808	103718.32648180598	103718.32648180597	4975.867988439353	98449.50006710076	292.95842626585545
target angle:  0.16135437771880848
0	135242	37245.2	37245.2	1	0
0.04738275080201189	107446.26346877216	107446.26346877214	8334.55980079033	98830.18977997602	281.513888005787
target angle:  0.20062617052890785
0	129804	30111.1	30111.1	1	0
0.06564316580882723	111658.01803811385	111658.01803811382	11832.471096723544	99572.48972459942	253.05721679084687
target angle:  0.23989796333900723
0	128587	24491.9	24491.9	1	0
Backtracking failed.
0.08809461147041164	116111.07660551966	116111.07660551966	15283.461109479429	100603.25826547845	224.35723056181192
target angle:  0.2791697561491066